In [10]:
video_file = 'drive/MyDrive/video1.mp4'

In [42]:
import face_recognition


In [ ]:
!pip install deepface

In [ ]:
!pip install fer

In [59]:
from fer import FER

In [60]:
emotion_detector = FER(mtcnn=True)

In [120]:
colour_sad = (51,51,205)
colour_happy = (0,255,0)
colour_neutral = (77,77,77)
dic_color = {'Angry': colour_sad, 'Sad': colour_sad, 'Neutral': colour_neutral,
             'Disgust': colour_sad, 'Surprise': colour_happy, 'Fear': colour_sad,
             'Happy': colour_happy, 'angry': colour_sad, 'sad': colour_sad, 'neutral': colour_neutral,
             'disgust': colour_sad, 'surprise': colour_happy, 'fear': colour_sad,
             'happy': colour_happy}

In [121]:
!rm -rf out-face
!mkdir out-face
!rm -rf out-faces
!mkdir out-faces
!rm -rf out
!mkdir out

In [122]:
import cv2
from keras.models import load_model
import numpy as np

classifier = load_model("drive/MyDrive/model.hdf5")
emotion_dict= {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}
label_map = dict((v,k) for k,v in emotion_dict.items()) 

vidcap = cv2.VideoCapture(video_file)
success,img = vidcap.read()

count = 0
dict_emotions = {}
while success:
  cv2.imwrite("out/frame%d.jpg" % count, img)

  face_locations = face_recognition.face_locations(img)
  if len(face_locations)>0:
    for each_face in face_locations:
      top, right, bottom, left = each_face
      face_image = img[top:bottom, left:right]
      dominant_emotion, emotion_score = emotion_detector.top_emotion(face_image)
      dict_emotions[count] = {'dominant_emotion': dominant_emotion, 'emotion_score': emotion_score}

      cv2.imwrite("out-face/frame%d.jpg" % count, face_image) 
      if dominant_emotion is not None:
        cv2.rectangle(img, (left, top), (right, bottom), dic_color[dominant_emotion], 2)
        cv2.putText(img, dominant_emotion, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    dic_color[dominant_emotion], 2)
      else:
        cv2.rectangle(img, (left, top), (right, bottom), colour_neutral, 2)

    cv2.imwrite("out-faces/frame%d.jpg" % count, img) 
  success,img = vidcap.read()
  count += 1


In [ ]:
import os.path as path
import os

image_folder = 'out-faces'


frame = cv2.imread(os.path.join('out', 'frame100.jpg'))
height, width, layers = frame.shape
size = (width, height)

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter('output.mp4', fourcc, 30, (width,height))

for i in range(450):
  if path.exists(os.path.join('out-faces', 'frame' + str(i) + '.jpg')):
    video.write(cv2.imread(os.path.join('out-faces', 'frame' + str(i) + '.jpg')))
  else:
    video.write(cv2.imread(os.path.join('out', 'frame' + str(i) + '.jpg')))

cv2.destroyAllWindows()
video.release()

In [100]:
!ls

drive				     out       out-faces    sample_data
haarcascade_frontalface_default.xml  out-face  project.avi
